In [28]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as ps
import requests
import json
import time
import os
import sys
import urllib.request
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,mean_squared_error
%matplotlib inline

In [12]:
df_1119 = pd.read_excel('../data/2011_2019_영화_데이터.xls')
df_20 = pd.read_excel('../data/2020_영화_데이터.xls')

In [13]:
df_1119.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11411 entries, 0 to 11410
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      11411 non-null  int64         
 1   영화명     11411 non-null  object        
 2   감독      11089 non-null  object        
 3   제작사     3160 non-null   object        
 4   수입사     8159 non-null   object        
 5   배급사     11396 non-null  object        
 6   개봉일     11411 non-null  datetime64[ns]
 7   영화유형    11411 non-null  object        
 8   영화형태    11410 non-null  object        
 9   국적      11411 non-null  object        
 10  전국스크린수  11411 non-null  int64         
 11  전국매출액   11411 non-null  int64         
 12  전국관객수   11411 non-null  int64         
 13  서울매출액   11405 non-null  float64       
 14  서울관객수   11411 non-null  int64         
 15  장르      11344 non-null  object        
 16  등급      11411 non-null  object        
 17  영화구분    11411 non-null  object        
dtypes: dat

# 주연배우, 평점, 평가자수, 상영시간 추가해야함
etc(트윗수,구글관심도)

스크린수 1000


In [16]:
df_1119.isnull().sum()

순번           0
영화명          0
감독         322
제작사       8251
수입사       3252
배급사         15
개봉일          0
영화유형         0
영화형태         1
국적           0
전국스크린수       0
전국매출액        0
전국관객수        0
서울매출액        6
서울관객수        0
장르          67
등급           0
영화구분         0
dtype: int64

In [66]:
df_1119[df_1119['국적']=='한국']

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
0,1,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,135748398910,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,2,극한직업,이병헌,"(주)어바웃필름,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,139647979516,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,3,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,115698654137,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,4,국제시장,윤제균,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,110828014630,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
6,7,베테랑,류승완,"(주)외유내강,(주)필름케이",NaN,(주)씨제이이엔엠,2015-08-05,개봉영화,장편,한국,1064,105024756250,13395400,2.676637e+10,3290366,액션,15세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11384,11385,3분 파트너-무삭제,박주영,스마일컨텐츠,NaN,스마일컨텐츠,2017-09-15,개봉영화,장편,한국,1,2000,1,0.000000e+00,0,멜로/로맨스,청소년관람불가,일반영화
11386,11387,5분 운동 30분 섹스,박은석,(주)씨맥스커뮤니케이션즈,NaN,(주)씨맥스커뮤니케이션즈,2019-09-27,개봉영화,장편,한국,1,1000,1,0.000000e+00,0,드라마,청소년관람불가,일반영화
11388,11389,50대 장모님의 불타는 사위사랑,김두민,두리컴,NaN,두리컴,2017-12-04,개봉영화,장편,한국,1,5000,1,0.000000e+00,0,기타,청소년관람불가,일반영화
11402,11403,P.S. 걸 무삭제,원석호,전망 PRODUCTION,NaN,전망 PRODUCTION,2017-02-06,개봉영화,장편,한국,1,5000,1,0.000000e+00,0,스릴러,청소년관람불가,일반영화


In [18]:
df_1119[df_1119['영화형태'].isnull()]

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
5963,5964,S 중독,카모가와 테츠로,NaN,(유)조이앤컨텐츠그룹,(유)조이앤컨텐츠그룹,2012-05-18,개봉영화,NaN,일본,1,350050,75,350050.0,75,성인물(에로),청소년관람불가,일반영화


In [42]:
df_1119['전국스크린수'].value_counts()

1       5305
20       613
5        612
2        224
4        159
        ... 
261        1
554        1
487        1
391        1
1587       1
Name: 전국스크린수, Length: 933, dtype: int64

In [43]:
df_1119[df_1119['전국관객수'] < 100]

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
5840,5841,소림사: 무림퇴마전,NaN,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2019-12-09,개봉영화,장편,중국,5,516000,99,300000.0,60,액션,12세이상관람가,독립/예술영화
5841,5842,야공작,다이 시지에,NaN,(주)액티버스엔터테인먼트,(주)액티버스엔터테인먼트,2017-06-01,개봉영화,장편,중국,1,705500,99,705500.0,99,멜로/로맨스,15세이상관람가,독립/예술영화
5842,5843,카마수트라 나잇,에릭 매닝,NaN,씨네힐,주식회사 마운틴픽쳐스,2013-01-10,개봉영화,장편,인도,2,626000,98,0.0,0,드라마,청소년관람불가,독립/예술영화
5843,5844,댄스 오브 드래곤,맥스 매닉스,NaN,(주)케이알씨지,(주)케이알씨지,2012-12-20,개봉영화,장편,싱가포르,1,610000,97,0.0,0,멜로/로맨스,12세이상관람가,독립/예술영화
5844,5845,메신저 2,마르틴 바르네비츠,NaN,㈜케이디미디어,㈜케이디미디어,2011-09-01,개봉영화,장편,미국,1,843000,97,843000.0,97,공포(호러),청소년관람불가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11406,11407,VJ의 은밀한 사생활,오카다 히로,NaN,(주)영진크리에이티브,(주)영진크리에이티브,2017-12-25,개봉영화,장편,일본,1,6000,1,0.0,0,멜로/로맨스,청소년관람불가,일반영화
11407,11408,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22,개봉영화,장편,일본,1,8000,1,0.0,0,성인물(에로),청소년관람불가,일반영화
11408,11409,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29,개봉영화,장편,한국,0,0,0,0.0,0,코미디,청소년관람불가,일반영화
11409,11410,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11,개봉영화,장편,베트남,0,0,0,0.0,0,액션,청소년관람불가,일반영화


# 전국스크린 1개이거나 관객 100명이하가 절반정도되는데 날려야하나?

In [29]:
df_1119[df_1119['감독'].isnull()].head(3)

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
777,778,러브 유어셀프 인 서울,NaN,"빅히트 엔터테인먼트,씨제이 씨지브이(CJ CGV)(주)",NaN,씨제이 씨지브이(CJ CGV)(주),2019-01-26,개봉영화,장편,한국,224,3209682900,342366,1.090468e+09,112697,공연,전체관람가,일반영화
1345,1346,카운트다운,NaN,NaN,(유)조이앤시네마,"(주)제이앤씨미디어그룹,TCO(주)더콘텐츠온",2019-12-12,개봉영화,장편,미국,238,611133000,73412,1.886565e+08,22405,공포(호러),15세이상관람가,일반영화
1525,1526,젝스키스 에이틴,NaN,(주)와이지엔터테인먼트,NaN,씨제이 씨지브이(CJ CGV)(주),2018-01-18,개봉영화,장편,한국,25,364405500,50419,2.258915e+08,29333,다큐멘터리,전체관람가,일반영화


In [38]:
df_1119['감독']

0                김한민
1                이병헌
2                김용화
3                윤제균
4        안소니 루소,조 루소
            ...     
11406         오카다 히로
11407        쿠도 마사노리
11408            허재형
11409          레 탄 선
11410            NaN
Name: 감독, Length: 11411, dtype: object

# 감독, 주연 배우 여러명일때 수치화 어떻게 해야할까?

김한민

이병헌

김한민, 이병헌   (sort 후 수치화?)

이병헌, 김한민

김한민, 김용화, 이병헌

In [41]:
df_1119['영화유형'].unique()

array(['개봉영화'], dtype=object)

# 네이버 영화 api에 관객수가 없음

In [35]:
client_id = "H2_6lcavpVyHW8211rUq"
client_secret = "D1XBjnKTe9"
encText = urllib.parse.quote('카운트다운')
url = f"https://openapi.naver.com/v1/search/movie.json?query="+encText+"&display=100"+"&yearfrom=2019&yearto=2019"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
res = response.read().decode("utf-8")
# dictionary = xmltodict.parse(res)
json_object = json.dumps(res,ensure_ascii=False) 
js = json.loads(json_object)
# df = pd.json_normalize(js['rss']['channel']['item'])

In [36]:
print(js)

{
"lastBuildDate": "Sat, 27 Nov 2021 19:02:29 +0900",
"total": 2,
"start": 1,
"display": 2,
"items": [
{
"title": "<b>카운트다운</b>",
"link": "https://movie.naver.com/movie/bi/mi/basic.nhn?code=190377",
"image": "https://ssl.pstatic.net/imgmovie/mdi/mit110/1903/190377_P05_161403.jpg",
"subtitle": "Countdown",
"pubDate": "2019",
"director": "저스틴 덱|",
"actor": "엘리자베스 라일|피터 파시넬리|탈리타 베이트먼|조던 캘러웨이|",
"userRating": "6.78"

},
{
"title": "크리미널 <b>카운트다운</b>",
"link": "https://movie.naver.com/movie/bi/mi/basic.nhn?code=210810",
"image": "https://ssl.pstatic.net/imgmovie/mdi/mit110/2108/210810_P01_151114.jpg",
"subtitle": "Criminal Countdown",
"pubDate": "2019",
"director": "에드가 마리|",
"actor": "",
"userRating": "0.00"

}
]
}



In [55]:
encText = urllib.parse.quote('카운트다운')
url = f'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=f5eef3421c602c6cb7ea224104795888&movieNm={encText}'
r = requests.get(url)
js = json.loads(r.text)

In [61]:
js

{'movieListResult': {'totCnt': 19,
  'source': '영화진흥위원회',
  'movieList': [{'movieCd': '20211082',
    'movieNm': '크리미널 카운트다운',
    'movieNmEn': 'Criminal Countdown',
    'prdtYear': '2019',
    'openDt': '',
    'typeNm': '장편',
    'prdtStatNm': '개봉예정',
    'nationAlt': '독일',
    'genreAlt': '스릴러,범죄,액션',
    'repNationNm': '독일',
    'repGenreNm': '스릴러',
    'directors': [],
    'companys': []},
   {'movieCd': '20190545',
    'movieNm': '카운트다운',
    'movieNmEn': 'Countdown',
    'prdtYear': '2019',
    'openDt': '20191212',
    'typeNm': '장편',
    'prdtStatNm': '개봉',
    'nationAlt': '미국',
    'genreAlt': '공포(호러),스릴러',
    'repNationNm': '미국',
    'repGenreNm': '공포(호러)',
    'directors': [],
    'companys': []},
   {'movieCd': '20183937',
    'movieNm': '카운트다운: 쇼크웨이브',
    'movieNmEn': 'SHOCKWAVE: COUNTDOWN TO DISASTER',
    'prdtYear': '2017',
    'openDt': '20181219',
    'typeNm': '장편',
    'prdtStatNm': '개봉',
    'nationAlt': '미국',
    'genreAlt': '액션',
    'repNationNm': '미국',
    

In [62]:
url2 = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=f5eef3421c602c6cb7ea224104795888&movieCd=20190545'
r2 = requests.get(url2)
js2 = json.loads(r2.text)

In [63]:
js2

{'movieInfoResult': {'movieInfo': {'movieCd': '20190545',
   'movieNm': '카운트다운',
   'movieNmEn': 'Countdown',
   'movieNmOg': '',
   'showTm': '90',
   'prdtYear': '2019',
   'openDt': '20191212',
   'prdtStatNm': '개봉',
   'typeNm': '장편',
   'nations': [{'nationNm': '미국'}],
   'genres': [{'genreNm': '공포(호러)'}, {'genreNm': '스릴러'}],
   'directors': [],
   'actors': [],
   'showTypes': [{'showTypeGroupNm': '2D', 'showTypeNm': '디지털'}],
   'companys': [{'companyCd': '20161101',
     'companyNm': '(주)제이앤씨미디어그룹',
     'companyNmEn': '',
     'companyPartNm': '배급사'},
    {'companyCd': '20139196',
     'companyNm': 'TCO(주)더콘텐츠온',
     'companyNmEn': 'THE CONTENTS ON',
     'companyPartNm': '배급사'},
    {'companyCd': '20142369',
     'companyNm': '(유)조이앤시네마',
     'companyNmEn': 'JOY N CINEMA',
     'companyPartNm': '수입사'}],
   'audits': [{'auditNo': '2019-MF02485', 'watchGradeNm': '15세이상관람가'}],
   'staffs': []},
  'source': '영화진흥위원회'}}

kobis 로 상영시간 가져올수있음 근데 배우 안나옴   -> 하루 3천회 제한

네이버영화 api 로 배우,평점 가져올수있음 근데 영화시간안나옴 -> 하루 2만5천회 제한

둘다 이용?

네이버 영화 페이지 크롤링?

https://movie.naver.com/movie/bi/mi/detail.naver?code=190377  

Q.1. 다음과 같은 문장을, C++, Java, Python 등의 객체지향 프로그래밍(OOP) 언어를 이용하여 프로그래밍 한다면, 메모리에 생성되는 객체(object)는 총 몇 개인가? (2점)

'철수와 혜진이는 버스를 타고 동물원에 가서 사자와 호랑이를 보았다'
2,3,5,6

In [68]:
A=10
B=10.5
C=50
D=20.8

In [69]:
C = A + B
D = A - B
C,D

(20.5, -0.5)

In [70]:
data = [ data for data in range(10) if data % 2 != 0 ]
data

[1, 3, 5, 7, 9]

In [71]:
x = [x**3 for x in range(4)]
print (x)

[0, 1, 8, 27]


In [72]:
np.array([1, 2, 3, 4]).reshape(-1, 2)

array([[1, 2],
       [3, 4]])

In [73]:
np.array([[1, 2], [3, 4]])

array([[1, 2],
       [3, 4]])

In [74]:
np.array([1, 2, 3, 4])[:3]

array([1, 2, 3])